In [143]:
import pandas as pd
import os

partidas = pd.read_csv(os.getcwd() + '/data/results.csv', parse_dates=['date'], dayfirst=True)
goals = pd.read_csv(os.getcwd() + '/data/goalscorers.csv', parse_dates=['date'], dayfirst=True)
shootouts = pd.read_csv(os.getcwd() + '/data/shootouts.csv', parse_dates=['date'], dayfirst=True)

partidas.set_index('date', inplace=True)

https://www.kaggle.com/code/pozz91/football-analysis-predicting-win-rate-from-goals

In [144]:
partidas.head()

,home_team,away_team,home_score,away_score,tournament,city,country,neutral
date,,,,,,,,
1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1873-03-08,England,Scotland,4,2,Friendly,London,England,False
1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
1875-03-06,England,Scotland,2,2,Friendly,London,England,False
1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


In [145]:
import numpy as np

result = []

for _, row in partidas.iterrows():
    if row['home_score'] > row['away_score']:
        result.append('vitoria')
    elif row['home_score'] < row['away_score']:
        result.append('derrota')
    else:
        result.append('empate')

partidas['result'] = result

In [146]:
partidas['gols'] = partidas['home_score'] + partidas['away_score']

In [147]:
#Definir a performance de todos as selecoes dentro de casa
selecoes_home = partidas.groupby(['home_team','result']).count()['country'].sort_values(ascending=False).reset_index().rename(columns={'country': 'partidas'})
selecoes_home.head()

### Brasil venceu 426 partidas em casa por exemplo

,home_team,result,partidas
0,Brazil,vitoria,426
1,Argentina,vitoria,376
2,Germany,vitoria,327
3,Mexico,vitoria,326
4,England,vitoria,326


In [148]:
#Definir a performance de todos as selecoes fora de casa
selecoes_away=partidas.groupby(['away_team','result']).count()['country'].sort_values(ascending=False).reset_index().rename(columns={'country': 'partidas'})
selecoes_away.replace({'derrota': 'vitoria', 'vitoria':'derrota'}, inplace=True)
selecoes_away.head()

### Brasil venceu 228 partidas fora de casa por exemplo
### Finlandia perdeu 252 partidas fora de casa por exemplo

,away_team,result,partidas
0,England,vitoria,271
1,Finland,derrota,252
2,Germany,vitoria,247
3,Uruguay,derrota,233
4,Brazil,vitoria,228


In [149]:
# Contando quantas partidas cada time jogou fora de casa
away_tot_matches=[]
for team in selecoes_away['away_team']:
    tot_matches= selecoes_away[selecoes_away['away_team'] == team]['partidas'].sum()
    away_tot_matches.append(tot_matches)

selecoes_away['away_matches']= away_tot_matches
selecoes_away['pct_victory_away'] = selecoes_away['partidas']/selecoes_away['away_matches']


# Contando quantas partidas cada time jogou em casa
home_matches=[]
for team in selecoes_home['home_team']:
    tot_matches= selecoes_home[selecoes_home['home_team']== team]['partidas'].sum()
    home_matches.append(tot_matches)
   
selecoes_home['home_matches']=home_matches
selecoes_home['pct_home_victory']= selecoes_home['partidas']/ selecoes_home['home_matches']

In [150]:
# Ajustando o index
selecoes_home.rename(columns={'result': 'home_resultados', 'partidas': 'home_partidas'}, inplace=True)
selecoes_home.set_index('home_team', inplace=True)

selecoes_home.head()

,home_resultados,home_partidas,home_matches,pct_home_victory
home_team,,,,
Brazil,vitoria,426,594,0.717172
Argentina,vitoria,376,570,0.659649
Germany,vitoria,327,526,0.621673
Mexico,vitoria,326,555,0.587387
England,vitoria,326,525,0.620952


In [151]:
# Ajustando o index
selecoes_away.rename(columns={'result': 'away_resultados', 'partidas': 'away_partidas'}, inplace=True)
selecoes_away.set_index('away_team', inplace=True)

selecoes_away.head()

,away_resultados,away_partidas,away_matches,pct_victory_away
away_team,,,,
England,vitoria,271,524,0.517176
Finland,derrota,252,433,0.581986
Germany,vitoria,247,460,0.536957
Uruguay,derrota,233,560,0.416071
Brazil,vitoria,228,427,0.533958


In [152]:
# Agrupando todas as vitorias em casa por seleção
home_winners= selecoes_home[selecoes_home['home_resultados'] == 'vitoria']

# Agrupando todas as derrotas em casa por seleção
home_losers= selecoes_home[selecoes_home['home_resultados'] =='derrota']

# Agrupando todas as vitorias fora de casa por seleção
away_winners= selecoes_away[selecoes_away['away_resultados'] =='vitoria']

# Agrupando todas as derrotas fora de casa por seleção
away_losers= selecoes_away[selecoes_away['away_resultados'] =='derrota']

In [153]:
winners=pd.merge(home_winners, away_winners, left_index=True, right_index=True, how='inner')
losers=pd.merge(home_losers, away_losers, left_index=True, right_index=True, how='inner')
losers.rename(columns={'pct_home_victory': 'pct_home_defeats', 'pct_victory_away': 'pct_away_defeats'}, inplace=True)

In [158]:
winners

,home_resultados,home_partidas,home_matches,pct_home_victory,away_resultados,away_partidas,away_matches,pct_victory_away
Brazil,vitoria,426,594,0.717172,vitoria,228,427,0.533958
Argentina,vitoria,376,570,0.659649,vitoria,175,448,0.390625
Germany,vitoria,327,526,0.621673,vitoria,247,460,0.536957
Mexico,vitoria,326,555,0.587387,vitoria,151,380,0.397368
England,vitoria,326,525,0.620952,vitoria,271,524,0.517176
...,...,...,...,...,...,...,...,...
Chameria,vitoria,1,1,1.000000,vitoria,1,4,0.250000
South Ossetia,vitoria,1,7,0.142857,vitoria,3,7,0.428571
Matabeleland,vitoria,1,2,0.500000,vitoria,2,6,0.333333
Biafra,vitoria,1,2,0.500000,vitoria,1,1,1.000000


In [159]:
winners['tot_count']= winners['home_partidas'] + winners['away_partidas']
winners['tot_matches']= winners['home_matches']  + winners['away_matches']

winners['tot_pct_victory']= winners['tot_count']/winners['tot_matches']
winners= winners[winners.tot_matches >= 100]
winners_pct= winners[['pct_home_victory', 'pct_victory_away', 'tot_pct_victory']]

In [160]:
winners_pct

,pct_home_victory,pct_victory_away,tot_pct_victory
Brazil,0.717172,0.533958,0.640548
Argentina,0.659649,0.390625,0.541257
Germany,0.621673,0.536957,0.582150
Mexico,0.587387,0.397368,0.510160
England,0.620952,0.517176,0.569113
...,...,...,...
Puerto Rico,0.275862,0.171429,0.218750
Seychelles,0.226415,0.042254,0.120968
Liechtenstein,0.094340,0.058824,0.075556
Andorra,0.100000,0.021277,0.061856


In [161]:
losers['tot_count']= losers['home_partidas'] + losers['away_partidas']
losers['tot_matches']= losers['home_matches'] + losers['away_matches']
losers['tot_pct_defeats']= losers['tot_count']/losers['tot_matches']
losers= losers[losers.tot_matches >= 100] #getting only clubs who have played at least 100 matches
losers_pct= losers[['pct_home_defeats', 'pct_away_defeats', 'tot_pct_defeats']]

C:\Users\guilh\AppData\Local\Temp\ipykernel_5524\3372216187.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  losers['tot_count']= losers['home_partidas'] + losers['away_partidas']
C:\Users\guilh\AppData\Local\Temp\ipykernel_5524\3372216187.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  losers['tot_matches']= losers['home_matches'] + losers['away_matches']
C:\Users\guilh\AppData\Local\Temp\ipykernel_5524\3372216187.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [162]:
losers_pct

,pct_home_defeats,pct_away_defeats,tot_pct_defeats
Luxembourg,0.719665,0.829016,0.768519
Finland,0.469565,0.581986,0.532134
Malta,0.617647,0.793651,0.695550
Northern Ireland,0.443077,0.580460,0.514116
Norway,0.339759,0.465394,0.402878
...,...,...,...
Montenegro,0.295775,0.507692,0.397059
Central African Republic,0.381818,0.795918,0.576923
Afghanistan,0.380952,0.569620,0.504132
Turkmenistan,0.258621,0.597701,0.462069


In [163]:
#total percentage
winners_pct.sort_values(by='tot_pct_victory', ascending=False)
winners_pct=np.round(winners_pct*100, 2)
winners_pct['tot_count']= winners.tot_count
winners_pct['tot_matches']= winners.tot_matches

In [164]:
winners_pct

,pct_home_victory,pct_victory_away,tot_pct_victory,tot_count,tot_matches
Brazil,71.72,53.40,64.05,654,1021
Argentina,65.96,39.06,54.13,551,1018
Germany,62.17,53.70,58.22,574,986
Mexico,58.74,39.74,51.02,477,935
England,62.10,51.72,56.91,597,1049
...,...,...,...,...,...
Puerto Rico,27.59,17.14,21.88,28,128
Seychelles,22.64,4.23,12.10,15,124
Liechtenstein,9.43,5.88,7.56,17,225
Andorra,10.00,2.13,6.19,12,194


In [165]:
losers_pct=np.round(losers_pct*100, 2)
losers_pct['tot_count']= losers.tot_count
losers_pct['tot_matches']= losers.tot_matches

In [166]:
losers_pct

,pct_home_defeats,pct_away_defeats,tot_pct_defeats,tot_count,tot_matches
Luxembourg,71.97,82.90,76.85,332,432
Finland,46.96,58.20,53.21,414,778
Malta,61.76,79.37,69.56,297,427
Northern Ireland,44.31,58.05,51.41,346,673
Norway,33.98,46.54,40.29,336,834
...,...,...,...,...,...
Montenegro,29.58,50.77,39.71,54,136
Central African Republic,38.18,79.59,57.69,60,104
Afghanistan,38.10,56.96,50.41,61,121
Turkmenistan,25.86,59.77,46.21,67,145


In [167]:
winners_pct.sort_values(by='tot_pct_victory', ascending=False)

,pct_home_victory,pct_victory_away,tot_pct_victory,tot_count,tot_matches
Brazil,71.72,53.40,64.05,654,1021
Germany,62.17,53.70,58.22,574,986
Spain,67.81,47.74,58.12,426,733
England,62.10,51.72,56.91,597,1049
Iran,63.27,45.89,55.62,292,525
...,...,...,...,...,...
Seychelles,22.64,4.23,12.10,15,124
Luxembourg,11.72,7.77,9.95,43,432
Somalia,13.33,5.97,8.93,10,112
Liechtenstein,9.43,5.88,7.56,17,225
